# Data Cleaning
1. Json format data for different banks is imported serially.
2. This data is cleaned for training
3. Dates of News published after 15:30 pm is changed to next day
4. News published on holidays is either accounted for the next day or next-next day
5. Finance data is called from yahoo API for min and max dates of the news published
6. A new feature involving just the Open-Close prices ('P/L') is created in the dataframe
7. There are usually 3 times more data for finance than news.
8. These finance data is reduced to only those that fall on the news date or between 1 or 2 coming days

In [598]:
import pandas as pd
import numpy as np
#import seaborn as sns
import json
import pprint
import datetime
import time
from nltk.corpus import stopwords
import string
from collections import defaultdict
#from datetime import timedelta  

In [599]:
# opening and loading json files for each bank separately and then saving their respective cleaned csv files
with open('State.json') as f:
    data = json.load(f)

In [600]:
# how the data looks now:
pprint.pprint(data['State'][0])

{'content': ['Banking sector during the week saw more of disappointing news of '
             'job cuts, bad loans and NPA divergences raising questions on '
             'bank auditors.',
             'But starting with some positive news for the weekend, loan '
             'borrowers would be happy to note that ',
             '\xa0 (SBI) cut its interest rates on ',
             '. This was announced a day after it cut marginal cost-based '
             'lending rates (MCLR), its ',
             ' and also slashed retail term deposit rates by 25 basis points.',
             'Additionally, it also tied-up with World Bank to ',
             '.',
             'On to some less happy news, non-performing asset (NPA) '
             'divergences in private sector banks raised ',
             '. This comes at a time when doing business in India is getting '
             'easier while banks disclose divergences in NPA classification '
             'for the second time in a row in FY17.',
  

In [601]:
# date parser
count=0
for i in data['State'][:]:
    #pprint.pprint(i['date'].strip())
    try:
        i['date']=datetime.datetime.strptime(i['date'].strip(), '%B %d, %Y').date()
        count+=1  
        #print(i['date'])
        #print(type(i['date']))
    except ValueError as e:
        print('ValueError:', e,'with date',i['date'])
        # pprint.pprint(i['date'][0:5])
print(count)

1008


In [602]:
for i in data['State'][:1]:
    print(type(i['date']))

<class 'datetime.date'>


In [603]:
# time parser
# changes the news date to the next day for news after 15:30 pm
count = 0
for i in data['State'][:]:
    
    try:
        i['time'] = i['time'].strip().split()[1]+' '+i['time'].strip().split()[2] # changes time in the format 7;30 pm 
        i['time'] = datetime.datetime.strptime(i['time'],'%I:%M %p').time()
        
        if i['time'] > datetime.datetime.strptime('15:30','%H:%M').time():
            #print('before date',i['date'])
            i['date']=i['date']+datetime.timedelta(days=1)
            #print('after date',i['date'])
        count+=1
       
    except (ValueError,IndexError) as e:
        print('IndexError:', e,'with time',i['time'])
print(count)
  

IndexError: list index out of range with time                                                                   
IndexError: list index out of range with time                                                                   
1006


In [604]:
def text_process(news):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all special characters like "\u00a0" 
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [word for word in news if word not in string.punctuation]
      
    #nopunc = [word for word in news if word not in ["\u00a0"] ]
    
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
   # print(nopunc)

    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [605]:
# content parser
# cleans the content
for i in data['State'][0:]:
    try:
        i['content']=text_process(i['content'])
        #print(text)
            
    except:
        print('\n*3 The error causing content',i['content'])

In [606]:
## check to see how the data looks!
for i in data['State'][0:3]:
    print('\n',i['time'])
    print('\n',i['date'])
    print('\n',i['content'])
    


 11:20:00

 2017-06-25

 [',', 'one', "world's", '50', 'largest', 'banks,', 'pays', 'small', 'fraction', 'top', 'management', 'compared', 'private', 'sector', 'players', 'like', 'Former', 'RBI', 'governor', 'Raghuram', 'Rajan', 'flagged', 'low', 'remuneration', 'issue', 'last', 'August', 'saying', 'makes', 'difficult', 'state-owned', 'banks', '"attract', 'top', 'talent,', 'especially', 'lateral', 'entry".According', 'annual', 'reports', 'various', 'banks,', 'SBI', 'chairman', 'Arundhati', 'Bhattacharya', 'took', 'home', 'Rs', '28.96', 'lakh', 'last', 'fiscal,', 'pittance', 'compared', 'remuneration', 'counterparts', 'private', 'banks', 'receive.In', 'comparision,', 'ICICI', 'Bank', 'MD', 'CEO', 'Chanda', 'Kochhar', 'received', 'basic', 'salary', 'Rs', '2.66', 'crore', 'last', 'fiscal', 'besides', 'Rs', '2.2', 'crore', 'performance', 'bonus.', 'addition,', 'received', 'allowances', 'perquisites', 'Rs', '2.43', 'crore.Similarly,', 'Shikha', 'Sharma,', 'MD', 'CEO', 'Axis', 'Bank,', 'took

In [607]:
# converting data to dictionary with key as date and value as news
news_dictionary=defaultdict(list)

for i in data['State'][:]:
    news_dictionary[i['date']].extend(i['content'])

print(len(news_dictionary.keys()))

645


In [608]:
# reference cell for empty dates
"""
print(news_dictionary[datetime.date(2015, 11, 13)])

empty_keys=[datetime.date(2015, 11, 13),
 datetime.date(2015, 9, 25),
 datetime.date(2015, 8, 3),
 datetime.date(2017, 3, 17),
 datetime.date(2017, 3, 31),
 datetime.date(2017, 3, 30),
 datetime.date(2017, 7, 24),
 datetime.date(2017, 5, 31),
 datetime.date(2017, 6, 20),
 datetime.date(2017, 6, 23)]

for i in empty_keys:
    news_dictionary.pop(i)
  
#len(news_dictionary[datetime.date(2017, 5, 31)])
"""

'\nprint(news_dictionary[datetime.date(2015, 11, 13)])\n\nempty_keys=[datetime.date(2015, 11, 13),\n datetime.date(2015, 9, 25),\n datetime.date(2015, 8, 3),\n datetime.date(2017, 3, 17),\n datetime.date(2017, 3, 31),\n datetime.date(2017, 3, 30),\n datetime.date(2017, 7, 24),\n datetime.date(2017, 5, 31),\n datetime.date(2017, 6, 20),\n datetime.date(2017, 6, 23)]\n\nfor i in empty_keys:\n    news_dictionary.pop(i)\n  \n#len(news_dictionary[datetime.date(2017, 5, 31)])\n'

In [609]:
# getting input values for the yahoo api
k=news_dictionary.keys()
max_date=max(k)
min_date=min(k)

In [610]:
# Financial data:
import pandas_datareader.data as web   # Package and modules for importing data; this code may change depending on pandas version
import datetime
 
# We will look at stock prices over these dates
start = min_date
end =  max_date
bank_name = 'StateBANK.NS' 

bank_fin = web.DataReader("SBIN.NS", "yahoo", start, end)
 
type(bank_fin)
#bank_fin[datetime.date(2019,1,4)]
#bank_fin.reset_index(inplace=True)
#bank_fin[bank_fin['Date']=='2019-01-14']

pandas.core.frame.DataFrame

In [611]:
#Bank data as df
bank_fin.head()
bank_fin['P/L']=bank_fin['Close']-bank_fin['Open']
bank_fin.reset_index(inplace=True)
print(len(bank_fin['P/L'].values))# number of P/L values

2087


In [612]:
# initializing for the next cell
news_date_list = list(news_dictionary.keys())
bank_date_list = list(bank_fin['Date'])

In [613]:
# Picking out P/L for those dates with news articles
# This accounts for news that has in the weekends too!
# News on sat and sun are corelated to P/L on Monday (Works for any other holidays with two days gap too)

P_L_dict = {}
test_list = []
try:
    for i in range(len(bank_fin['Date'])):
        if bank_fin['Date'][i].date() in news_date_list:
            #print(date_list[i])
            P_L_dict[bank_fin['Date'][i].date()] = bank_fin['P/L'][i]  

    for holiday_dates in news_date_list:
        if holiday_dates not in bank_fin.values:
            for i in range(len(bank_fin['P/L'])):
                if holiday_dates + datetime.timedelta(days=1) == bank_fin['Date'][i]:
                    P_L_dict[holiday_dates] = bank_fin['P/L'][i]
                    break
                elif holiday_dates + datetime.timedelta(days=2) == bank_fin['Date'][i]:
                    P_L_dict[holiday_dates] = bank_fin['P/L'][i]
                    break
        
except (IndexError,ValueError) as e:
    print('This is the error',e)
    

for i in sorted(P_L_dict)[:5]:
    print(i,':',P_L_dict[i])

print(len(P_L_dict))

2012-08-21 : 1.2299957275390625
2012-08-22 : 1.214996337890625
2012-08-23 : -0.220001220703125
2012-08-24 : -0.6150054931640625
2012-08-27 : -4.5749969482421875
630


In [614]:
# labeling the positive and negative news and creating DF:
df = pd.DataFrame()

for i in news_dictionary:
    try:
        if P_L_dict[i]>=0:
            df = df.append({'news': news_dictionary[i], 'label': int(1)}, ignore_index=True)
                

        elif P_L_dict[i]<0:
            df = df.append({'news': news_dictionary[i], 'label': int(0)}, ignore_index=True)
           
            
    except:
        print('This is date that is causing trouble',i )

print(df.head())
print(len(df))

This is date that is causing trouble 2017-10-20
This is date that is causing trouble 2017-09-30
This is date that is causing trouble 2013-11-15
This is date that is causing trouble 2013-08-09
This is date that is causing trouble 2015-09-25
This is date that is causing trouble 2019-03-29
This is date that is causing trouble 2019-04-19
This is date that is causing trouble 2020-05-01
This is date that is causing trouble 2020-10-02
This is date that is causing trouble 2020-04-04
This is date that is causing trouble 2020-04-10
This is date that is causing trouble 2019-08-31
This is date that is causing trouble 2019-10-19
This is date that is causing trouble 2019-10-26
This is date that is causing trouble 2019-08-10
   label                                               news
0    0.0  [,, one, world's, 50, largest, banks,, pays, s...
1    0.0  [rally, PSU, banking, stocks, last, one, year,...
2    1.0  [Thursday, reduced, National, Electronic, Fund...
3    1.0  [Sharmila, Joshi, Sharmilajosh

In [615]:
# checking the consistency of length of first news after all this processing
print(len(df['news'][0]))
for i in news_dictionary:
    print(len(news_dictionary[i]))
    break

247
247


In [616]:
# checking for the consistency in number of news after all these processing
print(len(news_dictionary))
print(len(news_date_list))
print(len(df)) # difference is because there are two news that does not correlate to finance even in the next two days

645
645
630


In [617]:
# removing empty news finally!
def join(news):
    joined_news = ' '.join(news)
    return joined_news

df['news']=df['news'].apply(join)

In [619]:
df['news'].replace('', np.nan, inplace=True)
df.dropna(subset= ["news"],inplace=True)
print(df.head())

   label                                               news
0    0.0  , one world's 50 largest banks, pays small fra...
1    0.0  rally PSU banking stocks last one year largely...
2    1.0  Thursday reduced National Electronic Funds Tra...
3    1.0  Sharmila Joshi Sharmilajoshi.com told CNBC-TV1...
4    0.0  markets created fresh all-time high week gone ...


In [620]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598 entries, 0 to 629
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   598 non-null    float64
 1   news    598 non-null    object 
dtypes: float64(1), object(1)
memory usage: 14.0+ KB


In [621]:
# saving the file:
df.to_csv('State.csv',index=False) 

In [622]:
# checking the loading of data:
df_loaded =pd.read_csv('State.csv')
print(df_loaded.head())


   label                                               news
0    0.0  , one world's 50 largest banks, pays small fra...
1    0.0  rally PSU banking stocks last one year largely...
2    1.0  Thursday reduced National Electronic Funds Tra...
3    1.0  Sharmila Joshi Sharmilajoshi.com told CNBC-TV1...
4    0.0  markets created fresh all-time high week gone ...


# End!
### The saved csv files for different banks from here imported into Model.ipynb and then concatenated,
### which is used for building the model.